# WISCS
> Words and images in shared conceptual space

In [1]:
from wiscs.simulate import DataGenerator
from wiscs.utils import set_params
from wiscs.viz import Plotter

import numpy as np
import matplotlib.pyplot as plt 

In [2]:
# set experiment parameters
params = set_params(return_empty=True)

Params must be a dictionary of dictionaries with the following keys:
 dict_keys(['word', 'image', 'n_participants', 'n_trials', 'variance'])

In [3]:
params = {
    "word": {
        "perceptual": 200,
        "conceptual": 230, 
        "task": 100
    }, 
    "image": {
        "perceptual": 215,
        "conceptual": 230, 
        "task": 130
    },
    "n_participants": 200,
    "n_trials": 30,
    "variance": 0.1
}

if set_params(params):
    print("Parameters set successfully")

Parameters set successfully


In [4]:
data = DataGenerator(params=params).generate(dist_type='combined').to_pandas()
data.head()

,trial,type,RT,modality,participant
0,0,same,0.329264,word,0
1,0,same,0.483752,image,0
2,0,diff,0.566773,word,0
3,0,diff,0.461014,image,0
4,0,same,0.583543,word,1
